## General information

In this kernel I work with data from Google QUEST Q&A Labeling competition.

In this challenge we work with... opinions. This could help Q&A systems, so let's try!

## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import spacy
import gensim
from scipy.stats import spearmanr
from nltk.tokenize import TweetTokenizer
import datetime
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import time
import re
pd.set_option('max_colwidth',400)
from sklearn.multioutput import MultiOutputRegressor
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, BatchNormalization
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, Masking, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.layers import InputSpec, Layer
from tensorflow.keras.optimizers import Adam
import scipy as sp
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping, ReduceLROnPlateau
import scipy as sp
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder
import re
import torch
import os
import random
import numpy as np

from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")

## Helper functions

In [ ]:
# added preprocessing from https://www.kaggle.com/wowfattie/3rd-place/data

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\n', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))

    return df

def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')



def build_matrix(embedding_path: str = '',
                 embedding_path_spellcheck: str = r'f:\embeddings\wiki-news-300d-1M\wiki-news-300d-1M.vec',
                 word_dict: dict = None, lemma_dict: dict = None, max_features: int = 100000,
                 embed_size: int= 300, ):
    spell_model = gensim.models.KeyedVectors.load_word2vec_format(embedding_path_spellcheck)
    words = spell_model.index2word
    w_rank = {}
    for i, word in enumerate(words):
        w_rank[word] = i
    WORDS = w_rank

    def P(word):
        "Probability of `word`."
        # use inverse of rank as proxy
        # returns 0 if the word isn't in the dictionary
        return - WORDS.get(word, 0)

    def correction(word):
        "Most probable spelling correction for word."
        return max(candidates(word), key=P)

    def candidates(word):
        "Generate possible spelling corrections for word."
        return (known([word]) or known(edits1(word)) or [word])

    def known(words):
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)

    def edits1(word):
        "All edits that are one edit away from `word`."
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(word):
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))

    def singlify(word):
        return "".join([letter for i, letter in enumerate(word) if i == 0 or letter != word[i - 1]])


    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path, encoding='utf-8'))

    nb_words = min(max_features, len(word_dict))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    unknown_words = []
    for word, i in word_dict.items():
        key = word
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.lower())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.upper())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.capitalize())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = ps.stem(key)
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lc.stem(key)
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = sb.stem(key)
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        word = lemma_dict[key]
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            continue
        if len(key) > 1:
            word = correction(key)
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[word_dict[key]] = embedding_vector
                continue
        unknown_words.append(key)

    print(f'{len(unknown_words) * 100 / len(word_dict):.4f}% words are not in embeddings')
    return embedding_matrix, nb_words, unknown_words


def get_word_lemma_dict(full_text: list = None, ):
    nlp = spacy.load('en_core_web_lg', disable=['parser','ner','tagger'])
    nlp.vocab.add_flag(lambda s: s.lower() in spacy.lang.en.stop_words.STOP_WORDS, spacy.attrs.IS_STOP)
    word_dict = {}
    word_index = 1
    lemma_dict = {}
    docs = nlp.pipe(full_text, n_threads = os.cpu_count())
    for doc in docs:
        for token in doc:
            if (token.text not in word_dict) and (token.pos_ is not "PUNCT"):
                word_dict[token.text] = word_index
                word_index += 1
                lemma_dict[token.text] = token.lemma_

    return lemma_dict, word_dict

## Loading and exploring the data

In [ ]:
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 500)
path = '/kaggle/input/google-quest-challenge'
sample_submission = pd.read_csv(f'{path}/sample_submission.csv')
test = pd.read_csv(f'{path}/test.csv')
train = pd.read_csv(f'{path}/train.csv')

In [ ]:
train.head()

So we have the following information:
- question and its title;
- answer;
- questioner and answerer names and page links;
- url of the question;
- some kind of category and host;

Also we have 30 targets. They are very different and describe different things.

In this kernel I'll work with the texts themselves, without using additional information.

In [ ]:
plt.hist(train['question_body'].apply(lambda x: len(x.split())));
plt.title('Distribution of number of words in questions');

In [ ]:
plt.hist(train['answer'].apply(lambda x: len(x.split())));
plt.title('Distribution of number of words in answers');

It is interesting that there are a lot of long questions/answers. And there are one which are exteremely wrong - we will need to be able to deal with it.

In [ ]:
text = ' '.join(train['question_body'].values)
text_trigrams = [i for i in ngrams(text.split(), 3)]
Counter(text_trigrams).most_common(30)

As expected people describe what they are doing and what would they like to be able to do.

In [ ]:
text = ' '.join(train['answer'].values)
text_trigrams = [i for i in ngrams(text.split(), 3)]
Counter(text_trigrams).most_common(30)

And in answers people explain what needs to be done.

In [ ]:
text = ' '.join(train['question_body'].values)
wordcloud = WordCloud(max_font_size=None, background_color='white',
                      width=1200, height=1000).generate(text)
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud)
plt.title('Top words in questions')
plt.axis("off")
plt.show()

In [ ]:
text = ' '.join(train['answer'].values)
wordcloud = WordCloud(max_font_size=None, background_color='white',
                      width=1200, height=1000).generate(text)
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud)
plt.title('Top words in answers')
plt.axis("off")
plt.show()

## A basic modelling

As a first attempt, let's try classical approaches: tf-idf with linear regression

### preparing the data

In [ ]:
train = clean_data(train, ['answer', 'question_body', 'question_title'])
test = clean_data(test, ['answer', 'question_body', 'question_title'])

In [ ]:
tokenizer = TweetTokenizer()
train_df, valid_df, y_train, y_valid = train_test_split(train, train[sample_submission.columns[1:]], test_size=0.1)

vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer.tokenize)
full_text = list(train['question_body'].values) + list(test['question_body'].values)
vectorizer.fit(full_text)
train_vectorized = vectorizer.transform(train_df['question_body'])
valid_vectorized = vectorizer.transform(valid_df['question_body'])
test_vectorized = vectorizer.transform(test['question_body'])

vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer.tokenize)
full_text = list(train['answer'].values) + list(test['answer'].values)
vectorizer.fit(full_text)
train_vectorized1 = vectorizer.transform(train_df['answer'])
valid_vectorized1 = vectorizer.transform(valid_df['answer'])
test_vectorized1 = vectorizer.transform(test['answer'])

X_train = sp.sparse.hstack((train_vectorized, train_vectorized1))
X_valid = sp.sparse.hstack((valid_vectorized, valid_vectorized1))
X_test = sp.sparse.hstack((test_vectorized, test_vectorized1))

I'm using `MultiOutputRegressor` which allows to model multiple continuous outputs.

In [ ]:
ridge = Ridge()
model = MultiOutputRegressor(ridge, n_jobs=-1)

In [ ]:
%%time
model.fit(X_train, y_train);

In [ ]:
valid_pred = model.predict(X_valid)

In [ ]:
score = 0
for i, col in enumerate(train[sample_submission.columns[1:]]):
    score += np.nan_to_num(spearmanr(y_valid[col], valid_pred[:, i]).correlation) / 30

In [ ]:
print(f'Mean spearman correlation: {score}')

Not bad, right?

In [ ]:
pred = np.nan_to_num(model.predict(X_test))

## DL approach

There are several steps in preparing the data:
- combine all the texts (title, question, answer) into one field and fit tokenizer of it;
- generate word and lemma dictionaries with spacy for further preprocessing;
- load embedding matrix.

`build_matrix` function loads embeddings from a path, loads additional embeddings and tries to get embeddings for as many words as possible.

In [ ]:
%%time
train['text'] = train['question_body'] + ' ' + train['answer'] + ' ' + train['question_title']
test['text'] = test['question_body'] + ' ' + test['answer'] + ' ' + test['question_title']

full_text = list(train['text'].values) + list(test['text'].values)

tk = Tokenizer(lower = True, filters='')
tk.fit_on_texts(full_text)
print(f'Words in dictionary: {len(tk.word_index)}')

embed_size=300
embedding_path = "/kaggle/input/fasttext-crawl-300d-2m/crawl-300d-2M.vec"

lemma_dict, word_dict = get_word_lemma_dict(full_text)

In [ ]:
%time
embedding_matrix, nb_words, unknown_words = build_matrix(embedding_path, '/kaggle/input/wikinews300d1mvec/wiki-news-300d-1M.vec', word_dict, lemma_dict,
                                              100000, embed_size)

Now we prepare the data itself:
- texts are tokenized and padded;
- host and category are encoded and will be used in the model;

Some ideas for data processing and for models are from this kernel: https://www.kaggle.com/xhlulu/quest-bigru-starter-code

In [ ]:
tk.word_index = {k: v for k, v in tk.word_index.items() if k in word_dict.keys()}
train['host'] = train['host'].apply(lambda x: x.split('.')[0] if len(x.split('.')) == 3 else '')
test['host'] = test['host'].apply(lambda x: x.split('.')[0] if len(x.split('.')) == 3 else '')
unique_hosts = list(set(train['host'].unique().tolist() + test['host'].unique().tolist()))
host_dict = {i + 1: e for i, e in enumerate(unique_hosts)}
host_dict_reverse = {v: k for k, v in host_dict.items()}

unique_categories = list(set(train['category'].unique().tolist() + test['category'].unique().tolist()))
category_dict = {i + 1: e for i, e in enumerate(unique_categories)}
category_dict_reverse = {v: k for k, v in category_dict.items()}
max_len = 500
max_len_title = 30
train_question_tokenized = pad_sequences(tk.texts_to_sequences(train['question_body']), maxlen = max_len)
train_answer_tokenized = pad_sequences(tk.texts_to_sequences(train['answer']), maxlen = max_len)
train_title_tokenized = pad_sequences(tk.texts_to_sequences(train['question_title']), maxlen = max_len_title)

test_question_tokenized = pad_sequences(tk.texts_to_sequences(test['question_body']), maxlen = max_len)
test_answer_tokenized = pad_sequences(tk.texts_to_sequences(test['answer']), maxlen = max_len)
test_title_tokenized = pad_sequences(tk.texts_to_sequences(test['question_title']), maxlen = max_len_title)
train_host = train['host'].apply(lambda x: host_dict_reverse[x]).values
train_category = train['category'].apply(lambda x: category_dict_reverse[x]).values

test_host = test['host'].apply(lambda x: host_dict_reverse[x]).values
test_category = test['category'].apply(lambda x: category_dict_reverse[x]).values

In [ ]:
folds = KFold(n_splits=5, random_state=42)
for fold_n, (train_index, valid_index) in enumerate(folds.split(train)):
    break
    
X_train = [train_question_tokenized[train_index], train_answer_tokenized[train_index], train_title_tokenized[train_index],
           train_category[train_index], train_host[train_index]]
X_valid = [train_question_tokenized[valid_index], train_answer_tokenized[valid_index], train_title_tokenized[valid_index],
           train_category[valid_index], train_host[valid_index]]
X_test = [test_question_tokenized, test_answer_tokenized, test_title_tokenized, test_category, test_host]

y = train[sample_submission.columns[1:]].values
y_train = y[train_index]
y_valid = y[valid_index]

## Model architecture

In [ ]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [ ]:
def build_model_deep(embedding_matrix: np.array, max_len_title: int = 30, max_len: int = 500,
                     gru_dim: int = 150, n_gru_layers: int = 2, units: int = 128, spatial_dr: int = 0.1,
                     kernel_size1: int = 3, kernel_size2: int = 2, dense_units: int = 128, dr: int = 0.1,
                     conv_size: int = 32, lr: int = 0.001, lr_d: int = 0.0):
    text_embedding = Embedding(
        *embedding_matrix.shape,
        weights=[embedding_matrix],
        trainable=False,
        mask_zero=True,
        name='text_embedding'
    )
    category_embedding = Embedding(
        input_dim=6,
        output_dim=3,
        name='category_embedding'
    )
    host_embedding = Embedding(
        input_dim=56,
        output_dim=28,
        name='host_embedding'
    )

    question_body_in = Input(shape=(max_len,))
    answer_inp = Input(shape=(max_len,))
    question_title_in = Input(shape=(max_len_title,))
    category_inp = Input(shape=(1,))
    host_inp = Input(shape=(1,))

    # question
    # x = Embedding(50001, embed_size, weights=[embedding_matrix], trainable=False)(question_body_in)
    x = text_embedding(question_body_in)
    x1 = SpatialDropout1D(spatial_dr)(x)

    x_gru = Bidirectional(GRU(units, return_sequences=True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x1)
    max_pool1_gru = GlobalMaxPooling1D()(x1)

    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool3_gru = GlobalAveragePooling1D()(x3)
    max_pool3_gru = GlobalMaxPooling1D()(x3)

    x_lstm = Bidirectional(LSTM(units, return_sequences=True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x1)
    max_pool1_lstm = GlobalMaxPooling1D()(x1)

    x3 = Conv1D(conv_size, kernel_size=kernel_size2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool3_lstm = GlobalAveragePooling1D()(x3)
    max_pool3_lstm = GlobalMaxPooling1D()(x3)

    # answer
    question_gru_layers = [
        Bidirectional(GRU(gru_dim, return_sequences=True), name=f'BiGRU_q_{i}')
        for i in range(1, n_gru_layers * 2)
    ]
    answer = gru_block(answer_inp, question_gru_layers, text_embedding, block_name='answer')

    # title

    title_gru_layers = [
        Bidirectional(GRU(gru_dim, return_sequences=True), name=f'BiGRU_t_{i}')
        for i in range(1, n_gru_layers)
    ]
    title = gru_block(question_title_in, title_gru_layers, text_embedding, block_name='title')

    category = category_embedding(category_inp)
    host = host_embedding(host_inp)

    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
                     avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm,
                     answer, title, Flatten()(category), Flatten()(host)])
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(dense_units, activation='relu')(x))
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(int(dense_units / 2), activation='relu')(x))
    x = Dense(30, activation="sigmoid")(x)
    model = Model(inputs=[question_body_in, answer_inp, question_title_in, category_inp, host_inp], outputs=x)
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr=lr, decay=lr_d), metrics=['mse'])

    return model

def build_model2(embedding_matrix, n_categories, n_hosts, output_shape,
                gru_dim=150, n_gru_layers=3, n_fc_layers=2, hidden_dim=400):
    text_embedding = Embedding(
        *embedding_matrix.shape,
        weights=[embedding_matrix],
        trainable=False,
        mask_zero=True,
        name='text_embedding'
    )
    category_embedding = Embedding(
        input_dim=n_categories + 1,
        output_dim=3,
        name='category_embedding'
    )
    host_embedding = Embedding(
        input_dim=n_hosts + 1,
        output_dim=28,
        name='host_embedding'
    )

    # inputs
    qb_in = Input(shape=(500,))
    a_in = Input(shape=(500,))
    qt_in = Input(shape=(30,))
    c_in = Input(shape=(1,))
    h_in = Input(shape=(1,))

    # Embed the host and category labels
    c = category_embedding(c_in)
    h = host_embedding(h_in)
    #c = Reshape((3, ), name='category_reshape')(c)
    #h = Reshape((28, ), name='host_reshape')(h)

    # Pass data through stacked bidirectional GRUs
    gru_layers = [
        Bidirectional(GRU(gru_dim, return_sequences=True), name=f'BiGRU{i}')
        for i in range(1, n_gru_layers +1)
    ]
    qb = gru_block(qb_in, gru_layers, text_embedding, block_name='qbody')
    a = gru_block(a_in, gru_layers, text_embedding, block_name='answer')
    qt = gru_block(qt_in, gru_layers, text_embedding, block_name='qtitle')

    # Concatenate pooled title/body/answer with embedded category/host
    hidden = Concatenate(name='combine_all')([qt, qb, a, Flatten() (c), Flatten() (h)])

    # Fully-connected layers
    for _ in range(n_fc_layers):
        hidden = Dense(hidden_dim, activation='relu')(hidden)
        hidden = BatchNormalization()(hidden)
        hidden = Dropout(0.1)(hidden)

    out = Dense(output_shape, activation='sigmoid', name='output')(hidden)

    model = Model(inputs=[qb_in, a_in, qt_in, c_in, h_in], outputs=[out])

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001, amsgrad=True), metrics=['mae'])

    return model


def gru_block(x_in, gru_layers, embedding, block_name, hidden_dim=200):
    x = embedding(x_in)
    x = SpatialDropout1D(0.2, name=f'{block_name}_sp_drop')(x)

    for gru in gru_layers:
        x = gru(x)

    x = GlobalMaxPooling1D(name=f'{block_name}_pool')(x)
    x = Dense(hidden_dim, activation='relu', name=f'{block_name}_fc')(x)
    x = BatchNormalization(name=f'{block_name}_bn')(x)
    x = Dropout(0.5, name=f'{block_name}_drop')(x)

    return x

In [ ]:
model1 = build_model2(
    embedding_matrix, 
    n_categories=len(category_dict),
    n_hosts=len(host_dict),
    output_shape=30,
    n_gru_layers=3
)
model1.summary()
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1,
                              save_best_only=True, mode="min")
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=3)

In [ ]:

history = model1.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_valid, y_valid),
                    verbose=2, callbacks=[check_point, early_stop])

In [ ]:
valid_pred = model1.predict(X_valid, batch_size = 1024, verbose = 0)
score = 0
for i, col in enumerate(train[sample_submission.columns[1:]]):
    score += np.nan_to_num(spearmanr(y_valid[:, i], valid_pred[:, i]).correlation) / 30

In [ ]:
print(score)

In [ ]:
prediction = np.nan_to_num(model1.predict(X_test, batch_size = 1024, verbose = 0))

In [ ]:
sample_submission[sample_submission.columns[1:]] = sigmoid(prediction)
sample_submission.to_csv('submission.csv', index=False)